# Example Seldon Core Deployments using Ksonnet


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [ksonnet client](https://ksonnet.io/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

When you have a running minikube cluster run:


In [1]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [2]:
!kubectl create namespace seldon

namespace/seldon created


In [3]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


## Install Seldon Core
Create a ksonnet app and install the prototypes from our registry.

In [4]:
!rm -rf my-ml-deployment && ks init my-ml-deployment 

INFO Using context "minikube" from kubeconfig file "/home/clive/.kube/config" 
INFO Creating environment "default" with namespace "seldon", pointing to "version:v1.10.0" cluster at address "https://192.168.39.151:8443" 
INFO Generating ksonnet-lib data at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/lib/ksonnet-lib/v1.10.0' 


In [5]:
!cd my-ml-deployment && \
    ks registry add seldon-core ../../seldon-core && \
    ks pkg install seldon-core/seldon-core@master && \
    ks generate seldon-core seldon-core --withApife=false --withAmbassador=true --namespace=seldon --withRbac=true

INFO Retrieved 16 files                           
INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/seldon-core.jsonnet' 


In [6]:
!cd my-ml-deployment && \
      ks apply default

INFO Applying serviceaccounts seldon.seldon       
INFO Creating non-existent serviceaccounts seldon.seldon 
INFO Applying clusterroles seldon-crd             
INFO Creating non-existent clusterroles seldon-crd 
INFO Applying roles seldon.seldon-local           
INFO Creating non-existent roles seldon.seldon-local 
INFO Applying rolebindings seldon.seldon          
INFO Creating non-existent rolebindings seldon.seldon 
INFO Applying clusterrolebindings seldon.seldon   
INFO Creating non-existent clusterrolebindings seldon.seldon 
INFO Applying deployments seldon.seldon-core-seldon-cluster-manager 
INFO Creating non-existent deployments seldon.seldon-core-seldon-cluster-manager 
INFO Applying deployments seldon.seldon-core-redis 
INFO Creating non-existent deployments seldon.seldon-core-redis 
INFO Applying services seldon.seldon-core-redis   
INFO Creating non-existent services seldon.seldon-core-redis 
INFO Applying customresourcedefinitions seldondeployments.machinelearning.seldon.io

In [7]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-ambassador

Waiting for deployment "seldon-core-seldon-cluster-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

Install gRPC modules for the prediction protos.

In [8]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

## Serve Single Model

In [9]:
!cd my-ml-deployment && \
    ks generate seldon-serve-simple-v1alpha2 mymodel --image seldonio/mock_classifier:1.0 && \
    ks apply default -c mymodel

INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/mymodel.jsonnet' 
INFO Applying seldondeployments seldon.mymodel    
INFO Creating non-existent seldondeployments seldon.mymodel 


In [10]:
!cd my-ml-deployment && \
    ks show default -c mymodel

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: mymodel
  name: mymodel
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: mymodel
  name: mymodel
  predictors:
  - annotations:
      predictor_version: v1
    componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: mymodel
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    graph:
      children: []
      endpoint:
        type: REST
      name: mymodel
      type: MODEL
    name: mymodel
    replicas: 1


In [11]:
!kubectl rollout status deploy/mymodel-mymodel-mymodel-0
!kubectl rollout status deploy/mymodel-mymodel-svc-orch

Waiting for deployment "mymodel-mymodel-mymodel-0" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-mymodel-0" successfully rolled out
Waiting for deployment "mymodel-mymodel-svc-orch" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-svc-orch" successfully rolled out


Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [13]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:mymodel-mymodel-mymodel-0 replicas:1 replicasAvailable:1] map[name:mymodel-mymodel-svc-orch replicas:1 replicasAvailable:1]] state:Available]

### Get predictions

In [14]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [15]:
rest_request_ambassador("mymodel",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "8l8rnutjkh6l9mt0mkrn43mgq9",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.08050863353754148]
    }
  }
}


#### gRPC Request

In [16]:
 grpc_request_ambassador("mymodel",API_AMBASSADOR)

meta {
  puid: "91mqdoqinhualdpfoh80dp77dh"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.0765174090177312
  }
}



## Adding Authentication
We will add the example authentication from the Ambassador tutorial.

In [17]:
!kubectl apply -f resources/ambassador-auth-service-setup.yaml -n seldon

service/example-auth created
deployment.extensions/example-auth created


** Need to wait until running before adding Ambassador config **

In [20]:
!kubectl get pods -n seldon

NAME                                                  READY   STATUS    RESTARTS   AGE
example-auth-776ddb4c48-8pqv4                         1/1     Running   0          13s
mymodel-mymodel-mymodel-0-76f8fd9746-xfsdt            1/1     Running   0          3m
mymodel-mymodel-svc-orch-7dd77ffbcf-h9gd9             1/1     Running   0          3m
seldon-core-ambassador-778c58bf5d-shjfb               2/2     Running   0          8m
seldon-core-redis-5c498fc545-472dt                    1/1     Running   0          8m
seldon-core-seldon-cluster-manager-55cdc6f6bb-lb5rm   1/1     Running   0          8m


In [21]:
!kubectl apply -f resources/ambassador-auth-service-config.yaml -n seldon

service/example-auth configured


Show failed request when auth is running

In [24]:
rest_request_ambassador("mymodel",API_AMBASSADOR)

401



Show successful request with auth

In [25]:
rest_request_ambassador_auth("mymodel","username","password",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "6mh1psj04qploev8icdko40ahk",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07768525331823416]
    }
  }
}


In [26]:
!kubectl delete -f resources/ambassador-auth-service-setup.yaml -n seldon

service "example-auth" deleted
deployment.extensions "example-auth" deleted


In [27]:
!cd my-ml-deployment && \
    ks delete default -c mymodel && \
    ks component rm mymodel

INFO Deleting seldondeployments seldon.mymodel    
INFO removing environment component                component-name=mymodel
INFO Removing component parameter references ...  
INFO Deleting component 'mymodel' at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/mymodel.jsonnet' 
INFO Successfully deleted component 'mymodel'     


## Serve AB Test

In [28]:
!cd my-ml-deployment && \
    ks generate seldon-abtest-v1alpha2 myabtest --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 && \
    ks apply default -c myabtest

INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/myabtest.jsonnet' 
INFO Applying seldondeployments seldon.myabtest   
INFO Creating non-existent seldondeployments seldon.myabtest 


In [29]:
!cd my-ml-deployment && \
    ks show default -c myabtest

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: myabtest
  name: myabtest
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: myabtest
  name: myabtest
  predictors:
  - componentSpecs:
    - metadata:
        labels:
          version: v2
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-1
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - metadata:
        labels:
          version: v2
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-2
          volumeMounts: []
        terminationGracePeriodSeconds: 1
        volumes: []
    graph:
      children:
      - children: []
        endpoint:
          type: RES

In [30]:
!kubectl rollout status deploy/myabtest-myabtest-svc-orch

Waiting for deployment "myabtest-myabtest-svc-orch" rollout to finish: 0 of 1 updated replicas are available...
deployment "myabtest-myabtest-svc-orch" successfully rolled out


In [32]:
!kubectl get seldondeployments myabtest -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[replicasAvailable:1 name:myabtest-myabtest-svc-orch replicas:1] map[name:myabtest-myabtest-classifier-1-0 replicas:1 replicasAvailable:1] map[name:myabtest-myabtest-classifier-2-1 replicas:1 replicasAvailable:1]] state:Available]

### Get predictions

In [33]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [34]:
rest_request_ambassador("myabtest",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "8r2k2mk83inj8quuj9dknj4ntm",
    "tags": {
    },
    "routing": {
      "random-ab-test": 1
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07549333931105738]
    }
  }
}


#### gRPC Request

In [35]:
 grpc_request_ambassador("myabtest",API_AMBASSADOR)

meta {
  puid: "r2lb009dncpcpv772lvumhu1qn"
  routing {
    key: "random-ab-test"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08038082714039557
  }
}



In [36]:
!cd my-ml-deployment && \
    ks delete default -c myabtest && \
    ks component rm myabtest

INFO Deleting seldondeployments seldon.myabtest   
INFO removing environment component                component-name=myabtest
INFO Removing component parameter references ...  
INFO Deleting component 'myabtest' at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/myabtest.jsonnet' 
INFO Successfully deleted component 'myabtest'    


## Serve Multi-Armed Bandit

In [37]:
!cd my-ml-deployment && \
    ks generate seldon-mab-v1alpha2 mymab --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 && \
    ks apply default -c mymab

INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/mymab.jsonnet' 
INFO Applying seldondeployments seldon.mymab      
INFO Creating non-existent seldondeployments seldon.mymab 


In [38]:
!cd my-ml-deployment && \
    ks show default -c mymab

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: mymab
  name: mymab
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: mymab
  name: mymab
  predictors:
  - componentSpecs:
    - metadata:
        labels:
          version: v1
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-1
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - metadata:
        labels:
          version: v2
      spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: classifier-2
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - spec:
        containers:
        - image: seldonio/mab_epsilo

In [40]:
!kubectl rollout status deploy/mymab-mymab-svc-orch
!kubectl rollout status deploy/mymab-mymab-eg-router-2

deployment "mymab-mymab-svc-orch" successfully rolled out
Waiting for deployment "mymab-mymab-eg-router-2" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymab-mymab-eg-router-2" successfully rolled out


In [42]:
!kubectl get seldondeployments mymab -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:mymab-mymab-svc-orch replicas:1 replicasAvailable:1] map[name:mymab-mymab-classifier-1-0 replicas:1 replicasAvailable:1] map[replicasAvailable:1 name:mymab-mymab-classifier-2-1 replicas:1] map[replicasAvailable:0 name:mymab-mymab-eg-router-2 replicas:1]] state:Creating]

### Get predictions

In [43]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [44]:
rest_request_ambassador("mymab",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "ftt4ejqgep6fb3i1p1mnkgoek6",
    "tags": {
    },
    "routing": {
      "eg-router": 0
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.09284052128881116]
    }
  }
}


#### gRPC Request

In [45]:
 grpc_request_ambassador("mymab",API_AMBASSADOR)

meta {
  puid: "67dul2gn2flntkgomsci29qnq3"
  routing {
    key: "eg-router"
    value: 1
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.09354460371551045
  }
}



In [46]:
!cd my-ml-deployment && \
    ks delete default -c mymab && \
    ks component rm mymab

INFO Deleting seldondeployments seldon.mymab      
INFO removing environment component                component-name=mymab
INFO Removing component parameter references ...  
INFO Deleting component 'mymab' at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/mymab.jsonnet' 
INFO Successfully deleted component 'mymab'       


## Serve Model with Outlier Detector

In [47]:
!cd my-ml-deployment && \
    ks generate seldon-outlier-detector-v1alpha2 myout --image seldonio/mock_classifier:1.0 && \
    ks apply default -c myout

INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/myout.jsonnet' 
INFO Applying seldondeployments seldon.myout      
INFO Creating non-existent seldondeployments seldon.myout 


In [48]:
!cd my-ml-deployment && \
    ks show default -c myout

---
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
    ksonnet.io/component: myout
  name: myout
  namespace: seldon
spec:
  annotations:
    deployment_version: v1
    project_name: myout
  name: myout
  predictors:
  - annotations:
      predictor_version: v1
    componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.0
          imagePullPolicy: IfNotPresent
          name: myout
          volumeMounts: []
        imagePullSecrets: []
        terminationGracePeriodSeconds: 1
        volumes: []
    - spec:
        containers:
        - image: seldonio/outlier_mahalanobis:0.3
          imagePullPolicy: IfNotPresent
          name: outlier-detector
        terminationGracePeriodSeconds: 1
    graph:
      children:
      - children: []
        endpoint:
          type: REST
        name: myout
        type: MODEL
      endpoint:
        type: REST
      name: outlier-detector
      type: TRA

In [49]:
!kubectl rollout status deploy/myout-myout-svc-orch
!kubectl rollout status deploy/myout-myout-outlier-detector-1

Waiting for deployment "myout-myout-svc-orch" rollout to finish: 0 of 1 updated replicas are available...
deployment "myout-myout-svc-orch" successfully rolled out
Waiting for deployment "myout-myout-outlier-detector-1" rollout to finish: 0 of 1 updated replicas are available...
deployment "myout-myout-outlier-detector-1" successfully rolled out


In [50]:
!kubectl get seldondeployments myout -o jsonpath='{.status}' -n seldon

map[state:Creating predictorStatus:[map[name:myout-myout-svc-orch replicas:1 replicasAvailable:1] map[name:myout-myout-myout-0 replicas:1 replicasAvailable:1] map[name:myout-myout-outlier-detector-1 replicas:1 replicasAvailable:0]]]

### Get predictions

In [51]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [52]:
rest_request_ambassador("myout",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "c7ss82cl61seqfkutm51b3043g",
    "tags": {
      "outlierScore": [0.0]
    },
    "routing": {
      "outlier-detector": -1
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.08623843760309204]
    }
  }
}


#### gRPC Request

In [53]:
 grpc_request_ambassador("myout",API_AMBASSADOR)

meta {
  puid: "3ts1sk25nheje0slibqrje3db4"
  tags {
    key: "outlierScore"
    value {
      list_value {
        values {
          number_value: 0.0
        }
      }
    }
  }
  routing {
    key: "outlier-detector"
    value: -1
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.09223083003575284
  }
}



In [54]:
!cd my-ml-deployment && \
    ks delete default -c myout && \
    ks component rm myout

INFO Deleting seldondeployments seldon.myout      
INFO removing environment component                component-name=myout
INFO Removing component parameter references ...  
INFO Deleting component 'myout' at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/myout.jsonnet' 
INFO Successfully deleted component 'myout'       


# Tear down

In [ ]:
!cd my-ml-deployment && ks delete default

In [ ]:
!rm -rf my-ml-deployment